In [1]:
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import tensorflow as tf

In [2]:
input_reader = JsonReader("logs2")

2022-12-14 15:20:35,240	WARNING json_reader.py:238 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs2/*.json', '/tf/Cage/Notebooks/logs2/*.zip']


In [19]:
sequence_length=5
num_episodes = 10000
windows_per_episode = 100-sequence_length
state_len = 52
num_action = 145
encoding_len = state_len + num_action

data_points = num_episodes * windows_per_episode

X = np.zeros((data_points,sequence_length,encoding_len))
Y = np.zeros((data_points,state_len))

index = 0
for e in range(num_episodes):
    data = input_reader.next()
    data['obs'].shape
    for i in range(0, windows_per_episode):
        ns = data['obs'][i+1]
        if i < sequence_length:
            s = np.zeros((sequence_length, encoding_len))
            for k in range(0,i):
                s[k,:] = np.concatenate([data['obs'][k], np.zeros(num_action)])
        else:
            vec = np.zeros((sequence_length, num_action))
            vec[np.arange(sequence_length),data['actions'][i-sequence_length:i]] = 1
            s = np.concatenate([data['obs'][i-sequence_length:i], vec], axis=1)
            
        X[index,:,:] = s
        Y[index,:] = ns
        index += 1
index

950000

In [ ]:
from keras.layers import Bidirectional
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import Bidirectional


model = Sequential()
model.add(Bidirectional(LSTM(255, activation='relu'), input_shape=(sequence_length, encoding_len)))
model.add(Dense(state_len, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X, Y, epochs=100, validation_split=0.1, verbose=1, callbacks=[callback])

Train on 855000 samples, validate on 95000 samples
Epoch 1/100
855000/855000 [==============================] - 265s 310us/sample - loss: 0.0711 - val_loss: 0.0654
Epoch 2/100
855000/855000 [==============================] - 265s 310us/sample - loss: 0.0666 - val_loss: 0.0648
Epoch 3/100
855000/855000 [==============================] - 266s 311us/sample - loss: 0.0656 - val_loss: 0.0646
Epoch 4/100
855000/855000 [==============================] - 264s 309us/sample - loss: 0.0649 - val_loss: 0.0648
Epoch 5/100
503424/855000 [================>.............] - ETA: 1:44 - loss: 0.0639

In [ ]:
print(model.predict(np.zeros((1,5,52)))[0])

In [ ]:
print(np.round(model.predict(np.zeros((1,5,52)))[0]))

In [208]:
X[5,:,:]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 

In [209]:
print(Y[5])

[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 0. 1.]


In [5]:
test = np.zeros((5,10))
test[np.array([0,1,2,3,4]),np.array([0,2,4,6,8])] = 1
test

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])